In [ ]:
import tensorflow as tf
from tensorflow import feature_column

print(tf.__version__)

!python -V

# Define categorical colunm for our text feature, which is preprocessed into sequence of tokens
text_column = feature_column.sequence_categorical_column_with_vocabulary_list(key='text', 
                                                                     vocabulary_list=list(['asd', 'asdf']))

text_embedding = feature_column.embedding_column(text_column, dimension=8)

# Then define the layers and model it self
# This example uses Keras Functional API instead of Sequential just for more generallity

# Define SequenceFeatures layer to pass feature_columns into Keras model
sequence_feature_layer = tf.keras.experimental.SequenceFeatures(text_embedding)

max_length = 6
# Define inputs for each feature column. See
# см. https://github.com/tensorflow/tensorflow/issues/27416#issuecomment-502218673
sequence_feature_layer_inputs = {}
sequence_feature_layer_inputs['text'] = tf.keras.Input(shape=(max_length,), name='text', dtype=tf.string)

sequence_feature_layer_outputs, _ = sequence_feature_layer(sequence_feature_layer_inputs)
print(sequence_feature_layer_outputs)

# Define outputs of SequenceFeatures layer 
# And accually use them as first layer of the model

# note here that SequenceFeatures layer produce tuple of two tensors as output. We need just first to pass next.
sequence_feature_layer_outputs, _ = sequence_feature_layer(sequence_feature_layer_inputs)

x = tf.keras.layers.Conv1D(8,4)(sequence_feature_layer_outputs)
x = tf.keras.layers.MaxPooling1D(2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x= tf.keras.layers.GlobalAveragePooling1D()(x)
# This example supposes binary classification, as labels are 0 or 1
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=[v for v in sequence_feature_layer_inputs.values()], outputs=x)

model.summary()

# This example supposes binary classification, as labels are 0 or 1
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              #run_eagerly=True
             )

model.save('model.h5')
model_loaded = tf.keras.models.load_model('model.h5')

model_loaded.summary()
# expected result: model_loaded is instantinated
# actual result: exception with text: "Unknown layer: SequenceFeatures"

#other way to save model structure also finishes with error

model_json_string = model.to_json()
# import pprint
# pprint.pprint(json.loads(model_json_string))
fresh_model = tf.keras.models.model_from_json(model_json_string)
fresh_model.summary()
# expected result: model_loaded is instantinated
# actual result: exception with text: "Unknown layer: SequenceFeatures"